**Group members: Jiahui Tang, Xiaohan Yang, Sarah Zeng**

# Lecture 21
## Tuesday, November 17th, 2020
### Databases II

# `SQLite` Exercises
Today you will work with the `candidates` and `contributors` datasets to create a database in `Python` using `SQLite`. This is a hands-on lecture. For once, you may do these tasks in a Jupyter notebook.

The exercises will consist of a sequence of steps to help illustrate basic commands.

<a id='deliverables'></a>
# Exercise Deliverables
1. Make a copy of this notebook and call it `L22_Exercises.ipynb.`
2. For each step in this lecture notebook, there are instructions labeled "**Do the following:**" (except for Step 1 and Interlude). Put all the code from those instructions in a single cell immediately following the instructions.  It should look like a `Python` script. You *must* comment where appropriate to demonstrate that you understand what you are doing.
   - **Note:** To get the pandas tables to display in a cell, use `display()`.
3. Save and close your database.  Be sure to upload your database with the lecture exercises. You must name your database **`L21DB.sqlite`**.

## Table of Contents
[Setting the Stage](#setting_the_stage)

[Step 1](#step_1)

[Interlude](#interlude): Not required but highly recommended.

[Step 2](#step_2)

[Step 3](#step_3)

[Step 4](#step_4)

[Step 5](#step_5)

[Step 6](#step_6)

[Step 7](#step_7)

[Step 8](#step_8)

---

<a id='setting_the_stage'></a>
# Setting the Stage
You should import `sqlite3` again like last time.

In [1]:
import sqlite3

We will also use a basic a `pandas` feature to display tables in the database.  Although this lecture isn't on `pandas`, I will still have you use it a little bit.

In [2]:
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

from IPython.display import display

Now we create the tables in the database (just like last time).

In [3]:
db = sqlite3.connect('L21DB.sqlite')
cursor = db.cursor()

cursor.execute("DROP TABLE IF EXISTS candidates")
cursor.execute("DROP TABLE IF EXISTS contributors")
cursor.execute("PRAGMA foreign_keys=1")

cursor.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_init TEXT, 
               party TEXT NOT NULL)''')

db.commit() # Commit changes to the database

cursor.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

db.commit()

<a id='step_1'></a>
# Step 1
Read `candidates.txt` and `contributors.txt` and insert their values into the respective tables.

In [4]:
with open ("candidates.txt") as candidates:
    next(candidates) # jump over the header
    for line in candidates.readlines():
        cid, first_name, last_name, middle_name, party = line.strip().split('|')
        vals_to_insert = (int(cid), first_name, last_name, middle_name, party)
        cursor.execute('''INSERT INTO candidates 
                  (id, first_name, last_name, middle_init, party)
                  VALUES (?, ?, ?, ?, ?)''', vals_to_insert)

In [5]:
with open ("contributors.txt") as contributors:
    next(contributors)
    for line in contributors.readlines():
        cid, last_name, first_name, middle_name, street_1, street_2, \
            city, state, zip_code, amount, date, candidate_id = line.strip().split('|')
        vals_to_insert = (last_name, first_name, middle_name, street_1, street_2, 
                          city, state, int(zip_code), amount, date, candidate_id)
        cursor.execute('''INSERT INTO contributors (last_name, first_name, middle_name, 
                           street_1, street_2, city, state, zip, amount, date, candidate_id) 
                           VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', vals_to_insert)

---

<a id='interlude'></a>
## Interlude
Now that you have values in the tables of the database, it would be convenient to be able to visualize those tables in some way.  We'll write a little helper function to accomplish this.

In [6]:
def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = dict()
    for i, col_name in enumerate(cols):
        framelist[col_name] = [row[i] for row in q]
    return pd.DataFrame.from_dict(framelist)

Here's how we can use our helper function.  It gives a pretty nice visualization of our table.  You should do the same thing with the `contributors` table.

In [7]:
candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")]
query = '''SELECT * FROM candidates'''
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party
0,16,Mike,Huckabee,,R
1,20,Barack,Obama,,D
2,22,Rudolph,Giuliani,,R
3,24,Mike,Gravel,,D
4,26,John,Edwards,,D
5,29,Bill,Richardson,,D
6,30,Duncan,Hunter,,R
7,31,Dennis,Kucinich,,D
8,32,Ron,Paul,,R
9,33,Joseph,Biden,,D


### Do the following:
Display the contributors table.

In [8]:
contributors_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]
query = '''SELECT * FROM contributors'''
viz_tables(contributors_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,1,Agee,Steven,,549 Laurel Branch Road,,Floyd,VA,24091,500.0,2007-06-30,16
1,2,Ahrens,Don,,4034 Rennellwood Way,,Pleasanton,CA,94566,250.0,2007-05-16,16
2,3,Ahrens,Don,,4034 Rennellwood Way,,Pleasanton,CA,94566,50.0,2007-06-18,16
3,4,Ahrens,Don,,4034 Rennellwood Way,,Pleasanton,CA,94566,100.0,2007-06-21,16
4,5,Akin,Charles,,10187 Sugar Creek Road,,Bentonville,AR,72712,100.0,2007-06-16,16
...,...,...,...,...,...,...,...,...,...,...,...,...
170,171,ABESHAUS,MERRILL,M.,1801 N. HEREFORD DRIVE,,FLAGSTAFF,AZ,860011121,120.0,2008-01-16,37
171,172,ABRAHAM,GEORGE,,P.O. BOX 1504,,LAKE CHARLES,LA,706021504,800.0,2008-01-17,37
172,173,ABRAHAMSON,PETER,J.,1030 W. ROSCOE STREET,,CHICAGO,IL,606572207,50.0,2008-01-25,37
173,174,ABRAHAM,SALEM,A.,P.O. BOX 7,,CANADIAN,TX,790140007,1000.0,2008-01-17,37


<a id='step_2'></a>
# Step 2:  Various Queries
We can query our database for entries with certain characteristics.  For example, we can query the `candidates` table for entries whose middle name fields are not empty.

In [9]:
query = '''SELECT * FROM candidates WHERE middle_init <> ""'''
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party
0,34,Hillary,Clinton,R.,D
1,39,Christopher,Dodd,J.,D
2,41,Fred,Thompson,D.,R


We can also see how many entries satisfy the query:

In [10]:
print("{} candidates have a middle initial.".format(viz_tables(candidate_cols, query).shape[0]))

3 candidates have a middle initial.


This used a trick on the Pandas table. You'll get to practice counting in SQL later.

### Do the following queries:
* Display the contributors where the state is "PA"
* Display the contributors where the amount contributed is greater than $\$1000.00$.
* Display the contributors from the state "UT" where the amount contributed is greater than $\$1000.00$.
* Display the contributors who didn't list their state
  - **Hint**:  Match `state` to the empty string
* Display the contributors from "WA" or "PA"
  - **Hint**:  You will need to use `IN ("WA", "PA")` in your `SELECT` statement.
* Display the contributors who contributed between $\$100.00$ and $\$200.00$.
  - **Hint**: You can use the `BETWEEN 100.00 and 200.00` clause.

In [11]:
# Display the contributors where the state is "PA"
query = "SELECT * FROM contributors WHERE state =='PA'"
display(viz_tables(contributors_cols, query))

# Display the contributors where the amount contributed is greater than \$1000.00$1000.00.
query = '''SELECT * FROM contributors WHERE amount > 1000'''
display(viz_tables(contributors_cols, query))

query = '''SELECT * FROM contributors WHERE state == "UT" AND amount > 1000'''
display(viz_tables(contributors_cols, query))

# Display the contributors who didn't list their state
query = '''SELECT * FROM contributors WHERE state == ""'''
display(viz_tables(contributors_cols, query))

# Display the contributors from "WA" or "PA"
query = '''SELECT * FROM contributors WHERE state == "WA" OR state == "PA"'''
display(viz_tables(contributors_cols, query))

# Display the contributors who contributed between \$100.00$100.00 and \$200.00$200.00
query = ''' SELECT * FROM contributors WHERE amount <= 200 AND amount >= 100'''
display(viz_tables(contributors_cols, query))

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,71,BUCKLEY,WALTER,W.,1635 COUNTRY ROAD,,BETHLEHEM,PA,180155718,-100.0,2008-03-05,22
1,72,BUCKLEY,MARJORIE,B.,1635 COUNTRY ROAD,,BETHLEHEM,PA,180155718,-100.0,2008-03-05,22
2,94,Raught,Philip,M,4714 Plum Way,,Pittsburgh,PA,15201,-1046.0,2008-04-21,32
3,95,Ferrara,Judith,D,1508 Waterford Road,,Yardley,PA,19067,-1100.0,2008-04-21,32
4,166,ABEL,JOHN,H.,422 THOMAS STREET,,BETHLEHEM,PA,180153316,200.0,2008-01-22,37


,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,6,Akin,Mike,,181 Baywood Lane,,Monticello,AR,71655,1500.0,2007-05-18,16
1,10,Allen,John D.,,1052 Cannon Mill Drive,,North Augusta,SC,29860,1300.0,2007-06-29,16
2,14,Altes,R.D.,,8600 Moody Road,,Fort Smith,AR,72903,2300.0,2007-06-21,16
3,16,Anthony,John,,211 Long Island Drive,,Hot Springs,AR,71913,2300.0,2007-06-12,16
4,22,Baker,David,,2550 Adamsbrooke Drive,,Conway,AR,72034,2300.0,2007-04-11,16
5,29,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,2300.0,2007-08-14,20
6,31,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,4600.0,2007-08-14,20
7,34,Buck,Blaine,M,45 Eaton Ave,,Camden,ME,48431752,2300.0,2007-09-30,20
8,46,Buchanan,John,,2025 NW 29th Rd,,Boca Raton,FL,334316303,1300.0,2007-08-09,20
9,136,ABRAMOWITZ,NIRA,,411 HARBOR ROAD,,SOUTHPORT,CT,68901376,2300.0,2007-09-14,35


,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,29,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,2300.0,2007-08-14,20
1,31,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,4600.0,2007-08-14,20


,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,126,BOURNE,TRAVIS,,LAGE KAART 77,,BRASSCHATT,,2930,-500.0,2008-11-20,35


,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,63,BURKE,SUZANNE,M.,3401 EVANSTON,,SEATTLE,WA,981038677,-700.0,2008-03-05,22
1,71,BUCKLEY,WALTER,W.,1635 COUNTRY ROAD,,BETHLEHEM,PA,180155718,-100.0,2008-03-05,22
2,72,BUCKLEY,MARJORIE,B.,1635 COUNTRY ROAD,,BETHLEHEM,PA,180155718,-100.0,2008-03-05,22
3,94,Raught,Philip,M,4714 Plum Way,,Pittsburgh,PA,15201,-1046.0,2008-04-21,32
4,95,Ferrara,Judith,D,1508 Waterford Road,,Yardley,PA,19067,-1100.0,2008-04-21,32
5,101,Aaronson,Rebecca,,2000 Village Green Dr Apt 12,,Mill Creek,WA,980125787,100.0,2008-02-08,34
6,107,Aaronson,Rebecca,,2000 Village Green Dr Apt 12,,Mill Creek,WA,980125787,100.0,2008-02-14,34
7,166,ABEL,JOHN,H.,422 THOMAS STREET,,BETHLEHEM,PA,180153316,200.0,2008-01-22,37


,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,4,Ahrens,Don,,4034 Rennellwood Way,,Pleasanton,CA,94566,100.0,2007-06-21,16
1,5,Akin,Charles,,10187 Sugar Creek Road,,Bentonville,AR,72712,100.0,2007-06-16,16
2,13,Allison,Rebecca,,3206 Summit Court,,Little Rock,AR,72227,200.0,2007-06-12,16
3,18,Arbogast,Robert,,12900 State Route 56 SE,,Mount Sterling,OH,43143,100.0,2007-06-22,16
4,28,Buckheit,Bruce,,8904 KAREN DR,,FAIRFAX,VA,220312731,100.0,2007-09-19,20
5,32,Buck,Thomas,,4206 Terrace Street,,Kansas City,MO,64111,100.0,2007-09-25,20
6,33,Buck,Jay,K.,1855 Old Willow Rd Unit 322,,Northfield,IL,600932918,200.0,2007-09-12,20
7,38,Bucher,Ida,M,1400 Warnall Ave,,Los Angeles,CA,900245333,100.0,2007-07-10,20
8,47,Buchanan,John,,2025 NW 29th Rd,,Boca Raton,FL,334316303,200.0,2007-08-14,20
9,101,Aaronson,Rebecca,,2000 Village Green Dr Apt 12,,Mill Creek,WA,980125787,100.0,2008-02-08,34


<a id='step_3'></a>
# Step 3:  Sorting
It could be beneficial to sort by one of the attributes in the database.  The following cell contains a basic sorting demo. Run it and try to understand what happened.

In [12]:
query = '''SELECT * FROM candidates ORDER BY id DESC'''
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party
0,41,Fred,Thompson,D.,R
1,39,Christopher,Dodd,J.,D
2,38,Tom,Tancredo,,R
3,37,John,McCain,,R
4,36,Samuel,Brownback,,R
5,35,Mitt,Romney,,R
6,34,Hillary,Clinton,R.,D
7,33,Joseph,Biden,,D
8,32,Ron,Paul,,R
9,31,Dennis,Kucinich,,D


### Do the following sorts on the `contributors` table:
* Sort the `candidates` table by `last_name`.
* Sort the contributors table by the `amount` in decending order where `amount` is restricted to be between $\$1000.00$ and $\$5000.00$.
    * **Hint:** In your SQL command, start with getting the amount between the specified range followed by the sort. This will all be done in one line.
* Sort the contributors who donted between $\$1000.00$ and $\$5000.00$ by `candidate_id` and then by `amount` in descending order.
  - **Hint**:  Multiple orderings can be accomplished by separating requests after `ORDER BY` with commas.
  - e.g. `ORDER BY amount ASC, last_name DESC`

In [13]:
#Sort the candidates table by last_name
query = '''SELECT * FROM contributors ORDER BY last_name '''
display(viz_tables(contributors_cols, query))
#Sort the contributors table by the amount in decending order where amount is 
#restricted to be between \$1000.00$1000.00 and \$5000.00$5000.00
query = '''SELECT * FROM contributors WHERE amount >= 1000 AND amount <=5000 ORDER BY amount DESC'''
display(viz_tables(contributors_cols, query))

#Sort the contributors who donted between \$1000.00$1000.00 and \$5000.00$5000.00 
#by candidate_id and then by amount in descending order.

query = ''' SELECT * FROM contributors WHERE amount <= 5000 AND amount >= 1000 ORDER BY candidate_id DESC, amount ASC'''
display(viz_tables(contributors_cols, query))

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,163,AARONS,CHARLES,,1730 SHORE DRIVE,,ANCHORAGE,AK,995153207,300.0,2008-01-30,37
1,164,AARONS,CHARLES,,1730 SHORE DRIVE,,ANCHORAGE,AK,995153207,410.0,2008-01-15,37
2,165,AARONS,CHARLES,,1730 SHORE DRIVE,,ANCHORAGE,AK,995153207,500.0,2008-01-09,37
3,162,ABACHERLI,SHIRLEY,M.,29875 NEWPORT ROAD,,MENIFEE,CA,925849524,150.0,2008-01-28,37
4,161,ABAIR,PETER,,40 EVANS STREET,,WATERTOWN,MA,24722150,25.0,2008-01-09,37
...,...,...,...,...,...,...,...,...,...,...,...,...
170,76,Schuff,Bryan,,1700 W Sweden Rd,,Brockport,NY,14420,-25.0,2008-08-22,32
171,128,TOLLESTRUP,TRAVIS,W.,16331 WINECREEK RD.,,SAN DIEGO,CA,92127,-1000.0,2008-05-15,35
172,85,Uihlein,Richard,,1396 N Waukegan Rd,,Lake Forest,IL,600451147,-2300.0,2008-04-21,32
173,84,Verster,Jeanette,M.,7220 SW 61st St,,Miami,FL,331431807,-1000.0,2008-04-21,32


,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,31,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,4600.0,2007-08-14,20
1,160,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,,MIAMI SPRINGS,FL,331663832,2600.0,2008-01-25,37
2,14,Altes,R.D.,,8600 Moody Road,,Fort Smith,AR,72903,2300.0,2007-06-21,16
3,16,Anthony,John,,211 Long Island Drive,,Hot Springs,AR,71913,2300.0,2007-06-12,16
4,22,Baker,David,,2550 Adamsbrooke Drive,,Conway,AR,72034,2300.0,2007-04-11,16
5,29,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,2300.0,2007-08-14,20
6,34,Buck,Blaine,M,45 Eaton Ave,,Camden,ME,48431752,2300.0,2007-09-30,20
7,136,ABRAMOWITZ,NIRA,,411 HARBOR ROAD,,SOUTHPORT,CT,68901376,2300.0,2007-09-14,35
8,6,Akin,Mike,,181 Baywood Lane,,Monticello,AR,71655,1500.0,2007-05-18,16
9,10,Allen,John D.,,1052 Cannon Mill Drive,,North Augusta,SC,29860,1300.0,2007-06-29,16


,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,157,ABBOTT,MIKE,E.,4516 OSPREY LNDG,,NICEVILLE,FL,325786810,1000.0,2008-01-15,37
1,174,ABRAHAM,SALEM,A.,P.O. BOX 7,,CANADIAN,TX,790140007,1000.0,2008-01-17,37
2,175,ABRAHAM,SALEM,A.,P.O. BOX 7,,CANADIAN,TX,790140007,1300.0,2008-01-30,37
3,160,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,,MIAMI SPRINGS,FL,331663832,2600.0,2008-01-25,37
4,139,ABOUBAKARE,NASAR,,1400 SAN MIGUEL DRIVE,,CORONA DEL MAR,CA,926251300,1000.0,2007-07-09,35
5,136,ABRAMOWITZ,NIRA,,411 HARBOR ROAD,,SOUTHPORT,CT,68901376,2300.0,2007-09-14,35
6,123,Aaron,Barbara,,2298 Pacific Ave # 6,,San Francisco,CA,941151435,1000.0,2008-02-11,34
7,45,Buchanan,John,,2025 NW 29th Rd,,Boca Raton,FL,334316303,1000.0,2007-08-28,20
8,46,Buchanan,John,,2025 NW 29th Rd,,Boca Raton,FL,334316303,1300.0,2007-08-09,20
9,29,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,2300.0,2007-08-14,20


<a id='step_4'></a>
# Step 4:  Selecting Columns
So far, we've been selecting all columns from a table (i.e. `SELECT * FROM`).  Often, we just want to select specific columns (e.g. `SELECT amount FROM`).

In [14]:
query = '''SELECT last_name, party FROM candidates'''
viz_tables(['last_name', 'party'], query)

,last_name,party
0,Huckabee,R
1,Obama,D
2,Giuliani,R
3,Gravel,D
4,Edwards,D
5,Richardson,D
6,Hunter,R
7,Kucinich,D
8,Paul,R
9,Biden,D


Using the `DISTINCT` clause, you remove duplicate rows.

In [15]:
query = '''SELECT DISTINCT party FROM candidates'''
viz_tables(['party'], query)

,party
0,R
1,D


### Do the following:
* Get the first and last name of contributors.  Make sure each row has distinct values.

In [16]:
query = '''SELECT DISTINCT first_name, last_name FROM contributors'''
display(viz_tables(['first_name', 'last_name'], query))

,first_name,last_name
0,Steven,Agee
1,Don,Ahrens
2,Charles,Akin
3,Mike,Akin
4,Rebecca,Akin
...,...,...
121,DENIS,ABERCROMBIE
122,MERRILL,ABESHAUS
123,GEORGE,ABRAHAM
124,PETER,ABRAHAMSON


<a id='step_5'></a>
# Step 5:  Altering Tables
The `ALTER` clause allows us to modify tables in our database.  Here, we add a new column to our candidates table called `full_name`.

In [17]:
cursor.execute('''ALTER TABLE candidates ADD COLUMN full_name TEXT''')
candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")]
viz_tables(candidate_cols, '''SELECT * FROM candidates''')

,id,first_name,last_name,middle_init,party,full_name
0,16,Mike,Huckabee,,R,None
1,20,Barack,Obama,,D,None
2,22,Rudolph,Giuliani,,R,None
3,24,Mike,Gravel,,D,None
4,26,John,Edwards,,D,None
5,29,Bill,Richardson,,D,None
6,30,Duncan,Hunter,,R,None
7,31,Dennis,Kucinich,,D,None
8,32,Ron,Paul,,R,None
9,33,Joseph,Biden,,D,None


What if we want to rename or delete a columm?  It can't be done with `SQLite` with a single command.  We need to follow some roundabout steps (see [`SQLite` ALTER TABLE](http://www.sqlitetutorial.net/sqlite-alter-table/)).  We won't consider this case at the moment.

For now, let's put a few commands together to populate the `full_name` column.

In [18]:
candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")] # regenerate columns with full_name
query = '''SELECT id, last_name, first_name FROM candidates''' # Select a few columns
full_name_and_id = [(attr[1] + ", " + attr[2], attr[0]) for attr in cursor.execute(query).fetchall()] # List of tuples: (full_name, id)

update = '''UPDATE candidates SET full_name = ? WHERE id = ?''' # Update the table
for rows in full_name_and_id:
    cursor.execute(update, rows)

query = '''SELECT * FROM candidates'''
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party,full_name
0,16,Mike,Huckabee,,R,"Huckabee, Mike"
1,20,Barack,Obama,,D,"Obama, Barack"
2,22,Rudolph,Giuliani,,R,"Giuliani, Rudolph"
3,24,Mike,Gravel,,D,"Gravel, Mike"
4,26,John,Edwards,,D,"Edwards, John"
5,29,Bill,Richardson,,D,"Richardson, Bill"
6,30,Duncan,Hunter,,R,"Hunter, Duncan"
7,31,Dennis,Kucinich,,D,"Kucinich, Dennis"
8,32,Ron,Paul,,R,"Paul, Ron"
9,33,Joseph,Biden,,D,"Biden, Joseph"


Here's another update, this time on an existing column.

In [19]:
update = '''UPDATE candidates SET full_name = "WINNER" WHERE last_name = "Obama"'''
cursor.execute(update)
update = '''UPDATE candidates SET full_name = "RUNNER-UP" WHERE last_name = "McCain"'''
cursor.execute(update)
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party,full_name
0,16,Mike,Huckabee,,R,"Huckabee, Mike"
1,20,Barack,Obama,,D,WINNER
2,22,Rudolph,Giuliani,,R,"Giuliani, Rudolph"
3,24,Mike,Gravel,,D,"Gravel, Mike"
4,26,John,Edwards,,D,"Edwards, John"
5,29,Bill,Richardson,,D,"Richardson, Bill"
6,30,Duncan,Hunter,,R,"Hunter, Duncan"
7,31,Dennis,Kucinich,,D,"Kucinich, Dennis"
8,32,Ron,Paul,,R,"Paul, Ron"
9,33,Joseph,Biden,,D,"Biden, Joseph"


### Do the following:
* Add a new column to the contributors table called `full_name`.  The value in that column should be in the form `last_name, first_name`.
* Change the value in the `full_name` column to the string `"Too Much"` if someone donated more than $\$1000.00$.

In [20]:
cursor.execute('''ALTER TABLE contributors ADD COLUMN full_name TEXT''')

contributor_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")] # regenerate columns with full_name
query = '''SELECT id, last_name, first_name FROM contributors''' # Select a few columns
full_name_and_id = [(attr[1] + ", " + attr[2], attr[0]) for attr in cursor.execute(query).fetchall()] # List of tuples: (full_name, id)

update = '''UPDATE contributors SET full_name = ? WHERE id = ?''' # Update the table
for rows in full_name_and_id:
    cursor.execute(update, rows)

query = '''SELECT * FROM contributors'''
display(viz_tables(contributor_cols, query))

update = '''UPDATE contributors SET full_name = 'Too Much' WHERE amount > 1000''' # Update the tablefor rows in full_name_and_id:
cursor.execute(update)

query = '''SELECT * FROM contributors WHERE amount > 1000'''
display(viz_tables(contributor_cols, query))



,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,full_name
0,1,Agee,Steven,,549 Laurel Branch Road,,Floyd,VA,24091,500.0,2007-06-30,16,"Agee, Steven"
1,2,Ahrens,Don,,4034 Rennellwood Way,,Pleasanton,CA,94566,250.0,2007-05-16,16,"Ahrens, Don"
2,3,Ahrens,Don,,4034 Rennellwood Way,,Pleasanton,CA,94566,50.0,2007-06-18,16,"Ahrens, Don"
3,4,Ahrens,Don,,4034 Rennellwood Way,,Pleasanton,CA,94566,100.0,2007-06-21,16,"Ahrens, Don"
4,5,Akin,Charles,,10187 Sugar Creek Road,,Bentonville,AR,72712,100.0,2007-06-16,16,"Akin, Charles"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,171,ABESHAUS,MERRILL,M.,1801 N. HEREFORD DRIVE,,FLAGSTAFF,AZ,860011121,120.0,2008-01-16,37,"ABESHAUS, MERRILL"
171,172,ABRAHAM,GEORGE,,P.O. BOX 1504,,LAKE CHARLES,LA,706021504,800.0,2008-01-17,37,"ABRAHAM, GEORGE"
172,173,ABRAHAMSON,PETER,J.,1030 W. ROSCOE STREET,,CHICAGO,IL,606572207,50.0,2008-01-25,37,"ABRAHAMSON, PETER"
173,174,ABRAHAM,SALEM,A.,P.O. BOX 7,,CANADIAN,TX,790140007,1000.0,2008-01-17,37,"ABRAHAM, SALEM"


,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,full_name
0,6,Akin,Mike,,181 Baywood Lane,,Monticello,AR,71655,1500.0,2007-05-18,16,Too Much
1,10,Allen,John D.,,1052 Cannon Mill Drive,,North Augusta,SC,29860,1300.0,2007-06-29,16,Too Much
2,14,Altes,R.D.,,8600 Moody Road,,Fort Smith,AR,72903,2300.0,2007-06-21,16,Too Much
3,16,Anthony,John,,211 Long Island Drive,,Hot Springs,AR,71913,2300.0,2007-06-12,16,Too Much
4,22,Baker,David,,2550 Adamsbrooke Drive,,Conway,AR,72034,2300.0,2007-04-11,16,Too Much
5,29,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,2300.0,2007-08-14,20,Too Much
6,31,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,4600.0,2007-08-14,20,Too Much
7,34,Buck,Blaine,M,45 Eaton Ave,,Camden,ME,48431752,2300.0,2007-09-30,20,Too Much
8,46,Buchanan,John,,2025 NW 29th Rd,,Boca Raton,FL,334316303,1300.0,2007-08-09,20,Too Much
9,136,ABRAMOWITZ,NIRA,,411 HARBOR ROAD,,SOUTHPORT,CT,68901376,2300.0,2007-09-14,35,Too Much


<a id='step_6'></a>
# Step 6: Aggregation
You can perform some nice operations on the values in the database.  For example, you can compute the maximum, minimum, and sum of a set.  You can also count the number of items in a given set.  Here's a little example.  You can do the rest.

In [21]:
contributor_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")] # You've already done this part. I just need to do it here b/c I haven't yet.
function = '''SELECT *, MAX(amount) AS max_amount FROM contributors'''
viz_tables(contributor_cols, function)


,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,full_name
0,31,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,4600.0,2007-08-14,20,Too Much


### Do the following:

* Modify the demo to only output the max amount.
    * **Hints:** 
        - Instead of using `SELECT *, MAX(amount) ...` you can try `SELECT MAX(amount) ...`
        - You will want to use `cursor.execute()` here and then `fetchall()`.
        - Do not display your results in a table. It is sufficient to write the answer out to the screen as a single number.
* Count how many donations there were above $\$1000.00$.
    * **Hint:** There is a `COUNT` function.
* Calculate the average *positive* donation.
    * **Hint:** There is an `AVG` function.
* Calculate the average contribution from each state and display in a table. Restrict to positive values again.
  - **Hint**:  Use code that looks like: 

  ```python
  "SELECT state,SUM(amount) FROM contributors GROUP BY state"
  ```

In [22]:
function = '''SELECT * FROM contributors WHERE amount < 0'''
viz_tables(contributor_cols, function)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,full_name
0,30,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,-2300.00,2007-08-14,20,"Buckel, Linda"
1,41,Buchanan,John,,2025 NW 29th Rd,,Boca Raton,FL,334316303,-500.00,2007-09-24,20,"Buchanan, John"
2,44,Buchanan,John,,2025 NW 29th Rd,,Boca Raton,FL,334316303,-700.00,2007-08-28,20,"Buchanan, John"
3,51,BYNUM,HERBERT,,332 SUNNYSIDE ROAD,,TAMPA,FL,336177249,-500.00,2008-03-10,22,"BYNUM, HERBERT"
4,52,BYINGTON,MARGARET,E.,2633 MIDDLEBORO LANE N.E.,,GRAND RAPIDS,MI,495061254,-2300.00,2008-03-03,22,"BYINGTON, MARGARET"
5,53,BYERS,BOB,A.,13170 TELFAIR AVENUE,,SYLMAR,CA,913423573,-2300.00,2008-03-07,22,"BYERS, BOB"
6,54,BYERS,AUDREY,,2658 LADBROOK WAY,,THOUSAND OAKS,CA,913615073,-200.00,2008-03-07,22,"BYERS, AUDREY"
7,55,BUSH,KRYSTIE,,P.O. BOX 61046,,DENVER,CO,802061046,-2300.00,2008-03-06,22,"BUSH, KRYSTIE"
8,56,BUSH,ERIC,,P.O. BOX 61046,,DENVER,CO,802061046,-2300.00,2008-03-06,22,"BUSH, ERIC"
9,57,BURTON,SUSAN,,9338 DEER CREEK DRIVE,,TAMPA,FL,336472286,-2300.00,2008-03-05,22,"BURTON, SUSAN"


In [23]:
contributor_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")] # You've already done this part. I just need to do it here b/c I haven't yet.
function = '''SELECT MAX(amount) AS max_amount FROM contributors'''
print(cursor.execute(function).fetchall())

# Count how many donations there were above \$1000.00$1000.00.
function = '''SELECT COUNT(amount) AS num_amount FROM contributors WHERE amount > 1000'''
print(cursor.execute(function).fetchall())

# Calculate the average positive donation
function = '''SELECT AVG(amount) AS avg_amount FROM contributors WHERE amount > 0'''
print(cursor.execute(function).fetchall())



#Calculate the average contribution from each state and display in a table. Restrict to positive values again.
function = '''SELECT AVG(amount) AS avg_amount, state FROM contributors WHERE amount > 0 GROUP BY state'''
display(viz_tables(['avg_amount', 'state'], function))


[(4600.0,)]
[(12,)]
[(480.6913675213675,)]


,avg_amount,state
0,403.333333,AK
1,1183.333333,AR
2,120.000000,AZ
3,284.053333,CA
4,2300.000000,CT
5,225.045000,DC
6,483.333333,FL
7,250.000000,IA
8,125.000000,IL
9,650.000000,LA


<a id='step_7'></a>
# Step 7: DELETE
We have already noted that `SQLite` can't drop columns in a straightfoward manner.  However, it can delete rows quite simply.  Here's the syntax:
```python
deletion = '''DELETE FROM table_name WHERE condition'''
```

### Do the following:
* Delete rows in the `contributors` table with last name "Ahrens".

In [24]:
deletion = '''DELETE FROM contributors WHERE last_name == 'Ahrens' '''
cursor.execute(deletion)
function = '''SELECT * FROM contributors WHERE last_name == 'Ahrens' '''
display(viz_tables(contributor_cols, function))

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,full_name


<a id='step_8'></a>
# Step 8:  LIMIT
The `LIMIT` clause offers convenient functionality.  It allows you to constrain the number of rows returned by your query.  It shows up in many guises.

In [25]:
query = '''SELECT * FROM candidates LIMIT 3'''
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party,full_name
0,16,Mike,Huckabee,,R,"Huckabee, Mike"
1,20,Barack,Obama,,D,WINNER
2,22,Rudolph,Giuliani,,R,"Giuliani, Rudolph"


In [26]:
query = '''SELECT * FROM candidates LIMIT 4 OFFSET 5'''
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party,full_name
0,29,Bill,Richardson,,D,"Richardson, Bill"
1,30,Duncan,Hunter,,R,"Hunter, Duncan"
2,31,Dennis,Kucinich,,D,"Kucinich, Dennis"
3,32,Ron,Paul,,R,"Paul, Ron"


In [27]:
query = '''SELECT * FROM candidates ORDER BY last_name LIMIT 4 OFFSET 5'''
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party,full_name
0,22,Rudolph,Giuliani,,R,"Giuliani, Rudolph"
1,24,Mike,Gravel,,D,"Gravel, Mike"
2,16,Mike,Huckabee,,R,"Huckabee, Mike"
3,30,Duncan,Hunter,,R,"Hunter, Duncan"


### Do the following:
* Query and display the ten most generous donors.
* Query and display the ten least generous donors who donated a positive amount of money (since the data we have has some negative numbers in it...).

In [28]:
query = '''SELECT * FROM contributors ORDER BY amount DESC LIMIT 10 '''
display(viz_tables(contributor_cols, query))


query = '''SELECT * FROM contributors WHERE amount >0 ORDER BY amount ASC LIMIT 10 '''
display(viz_tables(contributor_cols, query))

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,full_name
0,31,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,4600.0,2007-08-14,20,Too Much
1,160,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,,MIAMI SPRINGS,FL,331663832,2600.0,2008-01-25,37,Too Much
2,14,Altes,R.D.,,8600 Moody Road,,Fort Smith,AR,72903,2300.0,2007-06-21,16,Too Much
3,16,Anthony,John,,211 Long Island Drive,,Hot Springs,AR,71913,2300.0,2007-06-12,16,Too Much
4,22,Baker,David,,2550 Adamsbrooke Drive,,Conway,AR,72034,2300.0,2007-04-11,16,Too Much
5,29,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,2300.0,2007-08-14,20,Too Much
6,34,Buck,Blaine,M,45 Eaton Ave,,Camden,ME,48431752,2300.0,2007-09-30,20,Too Much
7,136,ABRAMOWITZ,NIRA,,411 HARBOR ROAD,,SOUTHPORT,CT,68901376,2300.0,2007-09-14,35,Too Much
8,6,Akin,Mike,,181 Baywood Lane,,Monticello,AR,71655,1500.0,2007-05-18,16,Too Much
9,10,Allen,John D.,,1052 Cannon Mill Drive,,North Augusta,SC,29860,1300.0,2007-06-29,16,Too Much


,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,full_name
0,27,Buckler,Steve,,24351 Armada Dr,,Dana Point,CA,926291306,25.0,2007-08-16,20,"Buckler, Steve"
1,50,Harrison,Ryan,,2247 3rd St,,La Verne,CA,917504918,25.0,2007-07-26,20,"Harrison, Ryan"
2,102,Aarons,Elaine,,481 Buck Island Rd Apt 17A,APT 17A,West Yarmouth,MA,26733300,25.0,2008-02-26,34,"Aarons, Elaine"
3,141,ABEGG,PATRICIA,T.,1862 E. 5150 S.,,SALT LAKE CITY,UT,841176911,25.0,2007-09-17,35,"ABEGG, PATRICIA"
4,144,ABEGG,PATRICIA,T.,1862 E. 5150 S.,,SALT LAKE CITY,UT,841176911,25.0,2007-08-06,35,"ABEGG, PATRICIA"
5,145,ABEGG,PATRICIA,T.,1862 E. 5150 S.,,SALT LAKE CITY,UT,841176911,25.0,2007-07-10,35,"ABEGG, PATRICIA"
6,161,ABAIR,PETER,,40 EVANS STREET,,WATERTOWN,MA,24722150,25.0,2008-01-09,37,"ABAIR, PETER"
7,159,ABBO,PAULINE,MORENCY,10720 JACOB LANE,,WHITE LAKE,MI,483862274,35.0,2008-01-07,37,"ABBO, PAULINE"
8,19,Ardle,William,,412 Dakota Avenue,,Springfield,OH,45504,50.0,2007-06-28,16,"Ardle, William"
9,26,Buckler,Steve,,24351 Armada Dr,,Dana Point,CA,926291306,50.0,2007-07-30,20,"Buckler, Steve"


# Save
Don't forget to save all of these changes to your database using `db.commit()`.  Before closing shop, be sure to close the database connection with `db.close()`.

In [29]:
db.commit()
db.close()